<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Load-the-Data" data-toc-modified-id="Load-the-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the Data</a></span><ul class="toc-item"><li><span><a href="#Define-and-create-a-Data-Loader" data-toc-modified-id="Define-and-create-a-Data-Loader-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Define and create a Data Loader</a></span></li><li><span><a href="#Visualize-some-images" data-toc-modified-id="Visualize-some-images-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Visualize some images</a></span></li></ul></li></ul></div>

# Libraries

In [11]:
import pickle as pkl
import matplotlib.pyplot as plt
import numpy as np
#import problem_unittests as tests

import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

%matplotlib inline

# Load the Data
Already pre-processed!

## Define and create a Data Loader

In [16]:
def get_dataloader(batch_size, image_size, data_dir='./data/processed_celeba_small/'):
    """
    Batch the neural network data using DataLoader
    :param batch_size: The size of each batch; the number of images in a batch
    :param img_size: The square size of the image data (x, y)
    :param data_dir: Directory where image data is located
    :return: DataLoader with batched data
    """
    
    # TODO: Implement function and return a dataloader
    
    # resize the images
    transform = transforms.Compose([transforms.Resize(image_size),
                                       transforms.ToTensor()])
    
    # define the datasets using ImageFolder
    dataset = datasets.ImageFolder(data_dir, transform)
    
    # create and return DataLoaders
    dL = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
    
    return dL

In [17]:
batch_size = 10
img_size = 32

# Call your function and get a dataloader
celeba_train_loader = get_dataloader(batch_size, img_size)

## Visualize some images